### --- Day 6: Chronal Coordinates ---
- Get the Points (call them destinations)
- From origin {0,0}, sort list into an ordereddict
- Define grid from the lowest to highest distinations from origin
- Create an object to represent each node that contains a destination flag and an array of nearest neighbors
- Iterate entire grid finding nearest destination to each point using Manhatten distance formula (if x=(a,b) and y=(c,d), then |a−c|+|b−d|)
- iterate entire grid, for each position add all unique nearest neighbors to a hashmap.
- display largest value

In [2]:
class Coordinate():
    def __init__(self, line):
        x = line.split(",")[0].strip()
        y = line.split(",")[1].strip()
        self.x = int(x)
        self.y = int(y)
    def __str__(self):
        return "[{0},{1}]".format(self.x,self.y)
    def __repr__(self):
        return self.__str__()

val1 = Coordinate("0,0")
val2 = Coordinate("1,1")
print(val1)
print(val2)

[0,0]
[1,1]


In [3]:
coords = open("6.txt", "r").readlines()
print(coords)

['1, 1\n', '1, 6\n', '8, 3\n', '3, 4\n', '5, 5\n', '8, 9']


In [59]:
def getManhattenDistanceBetween(val1,val2):
    if (not isinstance(val1,Coordinate) and not isinstance(val2,Coordinate)):
        print("Invalid input")
        return 
    #if debug: print(str(val1) + " & " + str(val2))
    return abs(val1.x-val2.x) + abs(val1.y-val2.y)

getManhattenDistanceBetween(Coordinate("0,0"),Coordinate("1, 1\n"))

2

In [60]:
from collections import OrderedDict 

def order_distinations1(coords):
    results = []
    for coord in coords:
        c = Coordinate(coord)
        dist_from_origin = getManhattenDistanceBetween(c,Coordinate('0,0'))
        result = (dist_from_origin, c)
        results.append(result)
        #print("[{0},{1}]::{2}".format(c.x,c.y,dist_from_origin))

    return OrderedDict(sorted(results, key=lambda x: x[0]))

ordered_distinations = order_distinations1(coords)
print(ordered_distinations)

OrderedDict([(2, [1,1]), (7, [3,4]), (10, [5,5]), (11, [8,3]), (17, [8,9])])


In [13]:
len(ordered_distinations)

5

In [32]:
last_key = list(ordered_distinations.keys())[-1]
print(ordered_distinations[last_key])

[8,9]


In [69]:
#inner to outer when building this map
dest_map = [[0 for (y) in range(ordered_distinations[last_key].x)] for i in range(ordered_distinations[last_key].x)]
dest_map

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [63]:
for id in ordered_distinations.keys():
    c = ordered_distinations[id]
    dest_map[c.x][c.y]=1  
dest_map

IndexError: list index out of range